## Figure 6: Climate projections (GCM and SSP)

In [ ]:
# spatial libraries
from shapely.geometry import Polygon
import rioxarray as rioxr
import geopandas as gpd
import xarray as xr
import xesmf as xe
import regionmask

from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import cftime
import os

# plotting libraries
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# colors
cl = px.colors.qualitative.D3

os.chdir('/home/rooda/Dropbox/Patagonia/')
period = slice("1980-01-01", "2019-12-31")

In [ ]:
# Basins
basins = gpd.read_file("GIS South/Basins_Patagonia_all.shp")
basins = basins.set_index("ID")

## Raw data

In [ ]:
os.chdir("/home/rooda/OGGM_results/Future_climate/")

lat_coords = np.arange(-56,-40, 0.5)
lon_coords = np.arange(-76,-67, 0.5)

baseline_period = slice("1980-01-01", "2010-01-01")
future_period   = slice("2070-01-01", "2100-01-01")

gcm_list  = ["ACCESS-CM2", "CMCC-ESM2", "EC-Earth3", "GFDL-ESM4", "INM-CM5-0", "KACE-1-0-G", "MPI-ESM1-2-HR", "MRI-ESM2-0", "MIROC6", "NorESM2-MM"]
ssp_list  = ['ssp126', 'ssp245', 'ssp585']

results_pp = []
results_t2m = []

for ssp in tqdm(ssp_list):
    
    results_gcm_pp  = []
    results_gcm_t2m = []
    
    for gcm in gcm_list:
        
        pp_model_ssp = xr.open_dataset("PP_" + gcm + "_" + ssp + ".nc")["pr"]
        pp_model_ssp = pp_model_ssp.interp(lat = lat_coords, lon = lon_coords)
        pp_model_ssp = pp_model_ssp.resample(time='YS').sum()
        pp_change    = (pp_model_ssp.sel(time = future_period).mean(dim="time") / pp_model_ssp.sel(time = baseline_period).mean(dim="time"))-1
        results_gcm_pp.append(pp_change)
        
        t2m_model_ssp = xr.open_mfdataset("T2M_" + gcm + "_" + ssp + ".nc")["tas"]
        t2m_model_ssp = t2m_model_ssp.interp(lat = lat_coords, lon = lon_coords)
        t2m_model_ssp = t2m_model_ssp.resample(time='YS').mean()
        t2m_change    = t2m_model_ssp.sel(time = future_period).mean(dim="time") - t2m_model_ssp.sel(time = baseline_period).mean(dim="time")
        results_gcm_t2m.append(t2m_change)
        
    results_gcm_pp  = xr.concat(results_gcm_pp,  dim='gcm')
    results_gcm_t2m = xr.concat(results_gcm_t2m, dim='gcm')
    results_pp.append(results_gcm_pp)
    results_t2m.append(results_gcm_t2m)
    
dataset = xr.merge([xr.concat(results_pp,  dim='ssp'), 
                    xr.concat(results_t2m, dim='ssp')])

In [ ]:
# select SSP 245 for map (a,b)
savg  = xe.SpatialAverager(dataset,  basins.geometry, geom_dim_name="avg")

basins["PP_change_126"] = savg(dataset.pr[0].mean(dim = "gcm"), skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PP_change_245"] = savg(dataset.pr[1].mean(dim = "gcm"), skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PP_change_585"] = savg(dataset.pr[2].mean(dim = "gcm"), skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values

basins["T2M_change_126"] = savg(dataset.tas[0].mean(dim = "gcm"), skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_change_245"] = savg(dataset.tas[1].mean(dim = "gcm"), skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_change_585"] = savg(dataset.tas[2].mean(dim = "gcm"), skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values

In [ ]:
# GCM uncertainty (SSP 245) > 80% of the models should agree the direction
gcm_spread = dataset.pr[1].where(dataset.pr[1] >= 0, 1).where(dataset.pr[1] < 0, -1).sum(dim = "gcm")
gcm_spread = gcm_spread.to_dataframe().reset_index()

gcm_spread = gpd.GeoDataFrame(gcm_spread.pr, geometry=gpd.points_from_xy(gcm_spread.lon,gcm_spread.lat))
gcm_spread = gcm_spread[(gcm_spread.pr >= 8) | (gcm_spread.pr <= -8)]

In [ ]:
# SSP uncertanty based on pertantage of the area
lat_coords = np.arange(-56,-40, 0.05) # reduce the resolution 
lon_coords = np.arange(-76,-67, 0.05)

dataset_hr = dataset.mean(dim = "gcm").interp(lat = lat_coords, lon = lon_coords)
dataset_hr = dataset_hr.where(regionmask.mask_geopandas(basins, dataset_hr) > 0, drop = True)

In [ ]:
geo_map = gpd.read_file("/home/rooda/Dropbox/ArcGIS/Chile/south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

poly_gdf = Polygon([(-76, -55.7), (-76, -40.52), (-68.05, -40.52), (-68.05, -55.7), (-76, -55.8)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)

geo_map = geo_map.clip(poly_gdf)

In [ ]:
fig = make_subplots(rows=2, cols=3, horizontal_spacing = 0.01, vertical_spacing = 0.1, column_widths = [0.34, 0.34, 0.32], 
                    subplot_titles = ["a) Precipitation (PP) change (%)","b) Temperature (T2M) change (ºC)","c) PP change by scenario (%)", "c) T2M change by scenario (ºC)"],
                    specs=[[{"type": "scattergeo", "rowspan": 2}, {"type": "scattergeo", "rowspan": 2}, {"type": "histogram"}],
                           [          None,                                      None,                  {"type": "histogram"}]])

# Precipitation mean change (a) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#d5d5d5", "#d5d5d5"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=1)

fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins.index, z = basins['PP_change_245']*100, 
                            colorscale = [(0.,"#fe7e0d"),(0.75, "#ffe9ba"),(1, "#1d78b4")], marker_line_color ='white', marker_line_width=0.1, 
                            zmin = -15, zmax = 5, colorbar=dict(len=0.45, x=0.24, y= 0.75, title='PP (mm)', thickness=20)), row=1, col=1)

fig.add_trace(go.Scattergeo(lat=gcm_spread.geometry.y, lon=gcm_spread.geometry.x, mode="markers", showlegend=False, 
                            marker=dict(color="black", size=1, opacity=0.5)), row=1, col=1)

# Temperature mean change (b) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#d5d5d5", "#d5d5d5"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=2)

cs = px.colors.sequential.Sunset
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['T2M_change_245'], 
                            colorscale=[cs[0], cs[3],  cs[5]], marker_line_color='white', marker_line_width=0.1, 
                            zmin = 1.2, zmax = 1.8, colorbar=dict(len=0.45, x=0.58, y= 0.75, title='T2M (ºC)', thickness=20)), row=1, col=2)

# layout a) and b)
fig.update_geos(showframe = True, framewidth = 1,  framecolor = "black", lonaxis_range=[-76, -68], lataxis_range=[-55.8, -40.5], 
                bgcolor = "#f9f9f9", showland = False, showcoastlines = False, showlakes = False)

# Precipitation ssp spread (c) -----------------------------------------------------------------------------------------------------------
pr_bins = dict(start=-25, size=0.5, end=7)
fig.add_trace(go.Histogram(x = dataset_hr.pr[0].values.flatten()*100,  marker_color = cl[0], histnorm='percent', xbins=pr_bins, name = "SSP126"), row=1, col=3)
fig.add_trace(go.Histogram(x = dataset_hr.pr[1].values.flatten()*100,  marker_color = cl[1], histnorm='percent', xbins=pr_bins, name = "SSP245"), row=1, col=3)
fig.add_trace(go.Histogram(x = dataset_hr.pr[2].values.flatten()*100,  marker_color = cs[5], histnorm='percent', xbins=pr_bins, name = "SSP585"), row=1, col=3)
fig.update_yaxes(title_text="Percent of total area (%)", side = "right", title_standoff = 2, row = 1, col = 3)
fig.update_traces(opacity=0.7, row = 1, col = 3)

# Temperature ssp spread (d) -----------------------------------------------------------------------------------------------------------
tas_bins = dict(start=0, size=0.04, end=3.5)
fig.add_trace(go.Histogram(x = dataset_hr.tas[0].values.flatten(), marker_color= cl[0], histnorm='percent', xbins=tas_bins, showlegend = False), row=2, col=3)
fig.add_trace(go.Histogram(x = dataset_hr.tas[1].values.flatten(), marker_color= cl[1], histnorm='percent', xbins=tas_bins, showlegend = False), row=2, col=3)
fig.add_trace(go.Histogram(x = dataset_hr.tas[2].values.flatten(), marker_color= cs[5], histnorm='percent', xbins=tas_bins, showlegend = False), row=2, col=3)
fig.update_yaxes(title_text="Percent of total area (%)", side = "right", title_standoff = 0, row = 2, col = 3)
fig.update_traces(opacity=0.7, row = 2, col = 3)

# layout c and d
fig.update_layout(barmode = "overlay",  plot_bgcolor="rgba(213,213,213,0.6)", legend = dict(yanchor="top", y=0.98, xanchor="left", x=0.70, bgcolor = 'rgba(255,255,255,0.5)'))
fig.update_xaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.5)", zeroline=False, showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)
fig.update_yaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.5)", zeroline=False, showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)

# general 
fig.update_layout(autosize = False, width = 900, height = 600, margin = dict(l=10, r=5, b=5, t=30, pad=0, autoexpand=True))
fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_6_GCM_SSP.png", scale=4)
fig.show()